<a href="https://colab.research.google.com/github/aanasingla/aana-the-techie/blob/main/Copy_of_Web_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests beautifulsoup4 fake-useragent pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.0 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin, urlparse
from fake_useragent import UserAgent
import pandas as pd

# Settings
start_url = "https://www.python.org"  # You can change this
email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
ua = UserAgent()
headers = {'User-Agent': ua.random}

# Storage
visited_urls = set()
emails_found = {}

# Categorize email by domain
def categorize_email(email):
    if "support" in email or "help" in email:
        return "Support"
    elif "info" in email or "contact" in email:
        return "Info"
    elif "edu" in email:
        return "Educational"
    else:
        return "General"

# Crawler Function
def crawl(url, depth=2):
    if depth == 0 or url in visited_urls:
        return
    visited_urls.add(url)

    try:
        print(f"Scanning: {url}")
        res = requests.get(url, headers=headers, timeout=5)
        soup = BeautifulSoup(res.text, "html.parser")

        # Extract and categorize emails
        new_emails = set(re.findall(email_regex, res.text))
        for email in new_emails:
            category = categorize_email(email)
            emails_found[email] = category

        # Crawl internal links
        for link in soup.find_all("a", href=True):
            next_url = urljoin(url, link['href'])
            if urlparse(next_url).netloc == urlparse(start_url).netloc:
                crawl(next_url, depth - 1)

    except Exception as e:
        print(f"Failed to fetch {url}: {e}")

In [3]:
# Start crawling
crawl(start_url, depth=2)

# Save to DataFrame
df = pd.DataFrame(list(emails_found.items()), columns=["Email", "Category"])
df.to_csv("scraped_emails.csv", index=False)

# Show result
print("\n📧 Emails Found:")
print(df)

# Optional download in Colab
from google.colab import files
files.download("scraped_emails.csv")

Scanning: https://www.python.org
Scanning: https://www.python.org#content
Scanning: https://www.python.org#python-network
Scanning: https://www.python.org/
Scanning: https://www.python.org/psf/
Scanning: https://www.python.org/jobs/
Scanning: https://www.python.org/community/
Scanning: https://www.python.org#top
Scanning: https://www.python.org#site-map
Scanning: https://www.python.org/community/irc/
Scanning: https://www.python.org/about/
Scanning: https://www.python.org/about/apps/
Scanning: https://www.python.org/about/quotes/
Scanning: https://www.python.org/about/gettingstarted/
Scanning: https://www.python.org/about/help/
Scanning: https://www.python.org/downloads/
Scanning: https://www.python.org/downloads/source/
Scanning: https://www.python.org/downloads/windows/
Scanning: https://www.python.org/downloads/macos/
Scanning: https://www.python.org/download/other/
Scanning: https://www.python.org/download/alternatives
Scanning: https://www.python.org/doc/
Scanning: https://www.pyt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>